In [2]:
import os
import yaml
from pathlib import Path

import pandas as pd

In [3]:
experiment_dirs = [
    "/home/kucerj56/FlashRAG/experiments/test_llama2_combined/output/",
    "/home/kucerj56/FlashRAG/experiments/test_llama2_meta_crag_post_chunk/output/",
    "/home/kucerj56/FlashRAG/experiments/test_llama31_combined/output/",
    "/home/kucerj56/FlashRAG/experiments/test_llama31_meta_crag_post_chunk/output/",
    "/home/kucerj56/FlashRAG/experiments/test_llama31_meta_crag_pre_chunk/output/",
    "/home/kucerj56/FlashRAG/experiments/test_llama31_nss_chonker/output/",
    "/home/kucerj56/FlashRAG/experiments/test_llama31_nss_chunked/output/",
    "/home/kucerj56/FlashRAG/experiments/test_llama31_nss_chunked_weaviate/output/",
    "/home/kucerj56/FlashRAG/experiments/test_llama31_nss_chunked_weaviate_w_entities/output/",
    "/home/kucerj56/FlashRAG/experiments/test_ministral_combined/output/",
    "/home/kucerj56/FlashRAG/experiments/test_ministral_meta_crag_post_chunk/output/",
    ]

In [4]:
all_exp_dirs = []
for d in experiment_dirs:
    print(f"Processing {d}")
    subdirs = [os.path.join(d, x) for x in os.listdir(d)]
    for sd in subdirs:
        if "generated.json" in os.listdir(sd):
            all_exp_dirs.append(sd)
        else:
            print(f"Skipping {sd}")
all_exp_dirs

Processing /home/kucerj56/FlashRAG/experiments/test_llama2_combined/output/
Processing /home/kucerj56/FlashRAG/experiments/test_llama2_meta_crag_post_chunk/output/
Processing /home/kucerj56/FlashRAG/experiments/test_llama31_combined/output/
Skipping /home/kucerj56/FlashRAG/experiments/test_llama31_combined/output/mmlu_2025_03_15_12_57_naive
Skipping /home/kucerj56/FlashRAG/experiments/test_llama31_combined/output/eli5_2025_03_16_02_31_crag
Skipping /home/kucerj56/FlashRAG/experiments/test_llama31_combined/output/arc_2025_03_16_02_33_crag
Skipping /home/kucerj56/FlashRAG/experiments/test_llama31_combined/output/hotpotqa_2025_03_16_02_52_crag
Skipping /home/kucerj56/FlashRAG/experiments/test_llama31_combined/output/2wikimultihopqa_2025_03_16_03_30_crag
Skipping /home/kucerj56/FlashRAG/experiments/test_llama31_combined/output/eli5_2025_03_16_17_08_crag
Skipping /home/kucerj56/FlashRAG/experiments/test_llama31_combined/output/arc_2025_03_16_17_14_crag
Skipping /home/kucerj56/FlashRAG/exper

['/home/kucerj56/FlashRAG/experiments/test_llama2_combined/output/mmlu_2025_03_15_15_17_zero-shot',
 '/home/kucerj56/FlashRAG/experiments/test_llama2_combined/output/arc_2025_03_15_15_24_zero-shot',
 '/home/kucerj56/FlashRAG/experiments/test_llama2_combined/output/hotpotqa_2025_03_15_15_25_zero-shot',
 '/home/kucerj56/FlashRAG/experiments/test_llama2_combined/output/mmlu_2025_03_15_15_30_naive',
 '/home/kucerj56/FlashRAG/experiments/test_llama2_combined/output/arc_2025_03_15_16_58_naive',
 '/home/kucerj56/FlashRAG/experiments/test_llama2_combined/output/hotpotqa_2025_03_15_17_20_naive',
 '/home/kucerj56/FlashRAG/experiments/test_llama2_combined/output/2wikimultihopqa_2025_03_15_17_54_naive',
 '/home/kucerj56/FlashRAG/experiments/test_llama2_combined/output/mmlu_2025_03_15_18_43_adaptive-rag',
 '/home/kucerj56/FlashRAG/experiments/test_llama2_combined/output/arc_2025_03_15_20_22_adaptive-rag',
 '/home/kucerj56/FlashRAG/experiments/test_llama2_combined/output/hotpotqa_2025_03_15_20_46_ad

In [5]:
len(all_exp_dirs)

367

In [6]:
def parse_results(results):
    result_dict = {}
    for line in results.strip().split("\n"):
        key, value = line.split(": ")
        if value == "None":
            result_dict[key] = None
        else:
            result_dict[key] = float(value)  # Convert value to float
    return result_dict

In [7]:
# conf['generator_model']
# conf['dataset_name']
# conf['split']
# conf['save_dir']

# # retrieval_model_path
# # generator_max_input_len


In [8]:
for exp_dir in all_exp_dirs:
    files = os.listdir(exp_dir)
    result_files = [f for f in files if f.startswith("metric_score")]
    # print(len(result_files))
    # sorted_result_files = sorted(result_files, reverse=True)
    # print(sorted_result_files)
    latest_result_file = sorted(result_files, reverse=True)[0]
    break

In [9]:
# TOOD add weaviate alpha in config

all_exp_df = None
for exp_dir in all_exp_dirs:
    with open(os.path.join(exp_dir, "config.yaml")) as f:
        conf = yaml.safe_load(f)
    if not any(["metric_score" in f for f in os.listdir(exp_dir)]):
        print(f"Skipping {exp_dir}")
        continue
    files = os.listdir(exp_dir)
    result_files = [f for f in files if f.startswith("metric_score")]
    latest_result_file = sorted(result_files, reverse=True)[0]
    with open(os.path.join(exp_dir, latest_result_file)) as f:
        results = f.read()
    print(f"parsing {exp_dir}/{latest_result_file}")
    result_dict = parse_results(results)
    if 'llm_judge_matcher_accuracy' not in result_dict:
        print(f"Missing LLM judge in {exp_dir}")
    df = pd.DataFrame([{
        'generator_model': conf['generator_model'],
        'dataset_name': conf['dataset_name'],
        'split': conf['split'],
        'save_dir': conf['save_dir'],
        'method': conf['save_note'],
        'corpus_path': conf['corpus_path'],
        'index_path': conf['index_path'],
        'weaviate_alpha': conf.get('weaviate_alpha', None),
        'use_weaviate': conf.get('retrieval_dense_use_weaviate', False),
        'retrieval_model_path': conf['retrieval_model_path'],
        'weaviate_query_properties': conf.get('weaviate_query_properties', False),
        'retrieval_topk': conf['retrieval_topk'],
        'weaviate_collection_name': conf.get('weaviate_collection_name'),
        'retrieved_document_prompt_properties': conf.get('retrieved_document_prompt_properties'),
        **result_dict
    }])
    if all_exp_df is None:
        all_exp_df = df
    else:
        all_exp_df = pd.concat([all_exp_df, df])

parsing /home/kucerj56/FlashRAG/experiments/test_llama2_combined/output/mmlu_2025_03_15_15_17_zero-shot/metric_score_2025_03_22_15_54.txt
parsing /home/kucerj56/FlashRAG/experiments/test_llama2_combined/output/arc_2025_03_15_15_24_zero-shot/metric_score_2025_03_22_15_57.txt
parsing /home/kucerj56/FlashRAG/experiments/test_llama2_combined/output/hotpotqa_2025_03_15_15_25_zero-shot/metric_score_2025_03_22_16_02.txt
parsing /home/kucerj56/FlashRAG/experiments/test_llama2_combined/output/mmlu_2025_03_15_15_30_naive/metric_score_2025_03_22_16_13.txt
parsing /home/kucerj56/FlashRAG/experiments/test_llama2_combined/output/arc_2025_03_15_16_58_naive/metric_score_2025_03_22_16_16.txt
parsing /home/kucerj56/FlashRAG/experiments/test_llama2_combined/output/hotpotqa_2025_03_15_17_20_naive/metric_score_2025_03_22_16_21.txt
parsing /home/kucerj56/FlashRAG/experiments/test_llama2_combined/output/2wikimultihopqa_2025_03_15_17_54_naive/metric_score_2025_03_22_16_30.txt
parsing /home/kucerj56/FlashRAG/e

/tmp/ipykernel_1554797/687047678.py:39: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_exp_df = pd.concat([all_exp_df, df])


parsing /home/kucerj56/FlashRAG/experiments/test_llama2_combined/output/mmlu_2025_03_16_01_53_self-rag/metric_score_2025_03_22_17_06.txt
parsing /home/kucerj56/FlashRAG/experiments/test_llama2_combined/output/arc_2025_03_16_02_53_self-rag/metric_score_2025_03_22_17_09.txt
parsing /home/kucerj56/FlashRAG/experiments/test_llama2_combined/output/hotpotqa_2025_03_16_03_08_self-rag/metric_score_2025_03_22_17_14.txt
parsing /home/kucerj56/FlashRAG/experiments/test_llama2_combined/output/2wikimultihopqa_2025_03_16_03_35_self-rag/metric_score_2025_03_22_17_21.txt
parsing /home/kucerj56/FlashRAG/experiments/test_llama2_combined/output/eli5_2025_03_16_14_56_zero-shot/metric_score_2025_03_22_17_24.txt
parsing /home/kucerj56/FlashRAG/experiments/test_llama2_combined/output/2wikimultihopqa_2025_03_16_14_58_zero-shot/metric_score_2025_03_22_17_31.txt
parsing /home/kucerj56/FlashRAG/experiments/test_llama2_combined/output/eli5_2025_03_16_15_01_naive/metric_score_2025_03_22_17_34.txt
parsing /home/kuc

/tmp/ipykernel_1554797/687047678.py:39: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_exp_df = pd.concat([all_exp_df, df])
/tmp/ipykernel_1554797/687047678.py:39: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_exp_df = pd.concat([all_exp_df, df])
/tmp/ipykernel_1554797/687047678.py:39: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes

parsing /home/kucerj56/FlashRAG/experiments/test_llama31_nss_chunked_weaviate/output/nss_2025_04_05_10_39_naive/metric_score_2025_04_05_16_45.txt
parsing /home/kucerj56/FlashRAG/experiments/test_llama31_nss_chunked_weaviate/output/nss_2025_04_05_18_11_naive/metric_score_2025_04_05_19_20.txt
parsing /home/kucerj56/FlashRAG/experiments/test_llama31_nss_chunked_weaviate/output/nss_2025_04_05_18_13_naive/metric_score_2025_04_05_19_21.txt
parsing /home/kucerj56/FlashRAG/experiments/test_llama31_nss_chunked_weaviate/output/nss_2025_04_05_18_15_naive/metric_score_2025_04_05_19_23.txt
parsing /home/kucerj56/FlashRAG/experiments/test_llama31_nss_chunked_weaviate/output/nss_2025_04_05_20_58_naive/metric_score_2025_04_05_21_10.txt
parsing /home/kucerj56/FlashRAG/experiments/test_llama31_nss_chunked_weaviate/output/nss_2025_04_05_21_00_naive/metric_score_2025_04_05_21_12.txt
parsing /home/kucerj56/FlashRAG/experiments/test_llama31_nss_chunked_weaviate/output/nss_2025_04_05_21_03_naive/metric_score

...

/tmp/ipykernel_1554797/687047678.py:39: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_exp_df = pd.concat([all_exp_df, df])
/tmp/ipykernel_1554797/687047678.py:39: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_exp_df = pd.concat([all_exp_df, df])
/tmp/ipykernel_1554797/687047678.py:39: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes

parsing /home/kucerj56/FlashRAG/experiments/test_ministral_combined/output/mmlu_2025_03_15_15_17_zero-shot/metric_score_2025_03_22_17_48.txt
parsing /home/kucerj56/FlashRAG/experiments/test_ministral_combined/output/arc_2025_03_15_15_23_zero-shot/metric_score_2025_03_22_17_51.txt
parsing /home/kucerj56/FlashRAG/experiments/test_ministral_combined/output/hotpotqa_2025_03_15_15_24_zero-shot/metric_score_2025_03_22_17_56.txt
parsing /home/kucerj56/FlashRAG/experiments/test_ministral_combined/output/2wikimultihopqa_2025_03_15_15_26_zero-shot/metric_score_2025_03_22_18_03.txt
parsing /home/kucerj56/FlashRAG/experiments/test_ministral_combined/output/mmlu_2025_03_15_15_29_naive/metric_score_2025_03_22_18_12.txt
parsing /home/kucerj56/FlashRAG/experiments/test_ministral_combined/output/arc_2025_03_15_16_11_naive/metric_score_2025_03_22_18_15.txt
parsing /home/kucerj56/FlashRAG/experiments/test_ministral_combined/output/hotpotqa_2025_03_15_16_21_naive/metric_score_2025_03_22_18_20.txt


/tmp/ipykernel_1554797/687047678.py:39: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_exp_df = pd.concat([all_exp_df, df])
/tmp/ipykernel_1554797/687047678.py:39: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_exp_df = pd.concat([all_exp_df, df])
/tmp/ipykernel_1554797/687047678.py:39: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes

parsing /home/kucerj56/FlashRAG/experiments/test_ministral_combined/output/2wikimultihopqa_2025_03_15_16_43_naive/metric_score_2025_03_22_18_28.txt
parsing /home/kucerj56/FlashRAG/experiments/test_ministral_combined/output/mmlu_2025_03_15_17_14_adaptive-rag/metric_score_2025_03_22_18_37.txt
parsing /home/kucerj56/FlashRAG/experiments/test_ministral_combined/output/arc_2025_03_15_18_14_adaptive-rag/metric_score_2025_03_22_18_40.txt
parsing /home/kucerj56/FlashRAG/experiments/test_ministral_combined/output/hotpotqa_2025_03_15_18_27_adaptive-rag/metric_score_2025_03_22_18_45.txt
parsing /home/kucerj56/FlashRAG/experiments/test_ministral_combined/output/2wikimultihopqa_2025_03_15_20_16_adaptive-rag/metric_score_2025_03_22_18_53.txt
parsing /home/kucerj56/FlashRAG/experiments/test_ministral_combined/output/eli5_2025_03_16_15_26_zero-shot/metric_score_2025_03_22_18_55.txt
parsing /home/kucerj56/FlashRAG/experiments/test_ministral_combined/output/eli5_2025_03_16_15_27_naive/metric_score_2025_

/tmp/ipykernel_1554797/687047678.py:39: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_exp_df = pd.concat([all_exp_df, df])
/tmp/ipykernel_1554797/687047678.py:39: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_exp_df = pd.concat([all_exp_df, df])
/tmp/ipykernel_1554797/687047678.py:39: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes

In [10]:
all_exp_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 354 entries, 0 to 0
Data columns (total 32 columns):
 #   Column                                                    Non-Null Count  Dtype  
---  ------                                                    --------------  -----  
 0   generator_model                                           354 non-null    object 
 1   dataset_name                                              354 non-null    object 
 2   split                                                     354 non-null    object 
 3   save_dir                                                  354 non-null    object 
 4   method                                                    354 non-null    object 
 5   corpus_path                                               354 non-null    object 
 6   index_path                                                354 non-null    object 
 7   weaviate_alpha                                            253 non-null    float64
 8   use_weaviate               

In [11]:
all_exp_df.to_csv("all_experiments_llm_nss2_last_w_property_0_8_exp_v10.csv", index=False)

In [ ]:
import os
import yaml

def update_yaml_metrics_in_directories(directories):
    new_metrics = ['avg_retrieval_calls', 'llm_judge']

    for dir_path in directories:
        config_file_path = os.path.join(dir_path, 'config.yaml')
        if os.path.isfile(config_file_path):
            try:
                # Load the YAML file
                with open(config_file_path, 'r') as file:
                    data = yaml.safe_load(file)

                # Check if 'metrics' key exists and is a list
                if isinstance(data, dict) and 'metrics' in data and isinstance(data['metrics'], list):
                    # Append new metrics if they are not already present
                    for metric in new_metrics:
                        if metric not in data['metrics']:
                            data['metrics'].append(metric)

                    # Write the updated data back to the file
                    with open(config_file_path, 'w') as file:
                        yaml.safe_dump(data, file, default_flow_style=False)

                    print(f"Updated metrics in {config_file_path}")
                else:
                    print(f"'metrics' key not found or is not a list in {config_file_path}")

            except Exception as e:
                print(f"Error processing {config_file_path}: {e}")
        else:
            print(f"config.yaml not found in {dir_path}")

# # Example usage
# all_exp_dirs = [
#     '/path/to/experiment1',
#     '/path/to/experiment2',
#     '/path/to/experiment3',
#     # Add more directories as needed
# ]

update_yaml_metrics_in_directories(all_exp_dirs)

Updated metrics in /home/kucerj56/FlashRAG/experiments/test_llama2_combined/output/mmlu_2025_03_15_15_17_zero-shot/config.yaml
Updated metrics in /home/kucerj56/FlashRAG/experiments/test_llama2_combined/output/arc_2025_03_15_15_24_zero-shot/config.yaml
Updated metrics in /home/kucerj56/FlashRAG/experiments/test_llama2_combined/output/hotpotqa_2025_03_15_15_25_zero-shot/config.yaml
Updated metrics in /home/kucerj56/FlashRAG/experiments/test_llama2_combined/output/mmlu_2025_03_15_15_30_naive/config.yaml
Updated metrics in /home/kucerj56/FlashRAG/experiments/test_llama2_combined/output/arc_2025_03_15_16_58_naive/config.yaml
Updated metrics in /home/kucerj56/FlashRAG/experiments/test_llama2_combined/output/hotpotqa_2025_03_15_17_20_naive/config.yaml
Updated metrics in /home/kucerj56/FlashRAG/experiments/test_llama2_combined/output/2wikimultihopqa_2025_03_15_17_54_naive/config.yaml
Updated metrics in /home/kucerj56/FlashRAG/experiments/test_llama2_combined/output/mmlu_2025_03_15_18_43_adapt

In [ ]:
import yaml

def update_yaml_metrics(file_path):
    try:
        # Load the YAML file
        with open(file_path, 'r') as file:
            data = yaml.safe_load(file)

        # Check if 'metrics' key exists and update the list
        if 'metrics' in data and isinstance(data['metrics'], list):
            new_metrics = ['avg_retrieval_calls', 'llm_judge']
            for metric in new_metrics:
                if metric not in data['metrics']:
                    data['metrics'].append(metric)

            # Write the updated data back to the file
            with open(file_path, 'w') as file:
                yaml.dump(data, file, default_flow_style=False)

            print(f"Updated metrics successfully in {file_path}")

        else:
            print(f"'metrics' key not found or not a list in {file_path}")

    except Exception as e:
        print(f"Error updating YAML file: {e}")

# Example usage
yaml_file_path = "/home/kucerj56/FlashRAG/experiments/test_llama2_combined/output/2wikimultihopqa_2025_03_15_17_54_naive/config.yaml"  # Replace with your actual YAML file path
update_yaml_metrics(yaml_file_path)

Updated metrics successfully in /home/kucerj56/FlashRAG/experiments/test_llama2_combined/output/2wikimultihopqa_2025_03_15_17_54_naive/config.yaml


In [ ]:
all_exp_dirs

['/home/kucerj56/FlashRAG/experiments/test_llama2_combined/output/mmlu_2025_03_15_15_17_zero-shot',
 '/home/kucerj56/FlashRAG/experiments/test_llama2_combined/output/arc_2025_03_15_15_24_zero-shot',
 '/home/kucerj56/FlashRAG/experiments/test_llama2_combined/output/hotpotqa_2025_03_15_15_25_zero-shot',
 '/home/kucerj56/FlashRAG/experiments/test_llama2_combined/output/mmlu_2025_03_15_15_30_naive',
 '/home/kucerj56/FlashRAG/experiments/test_llama2_combined/output/arc_2025_03_15_16_58_naive',
 '/home/kucerj56/FlashRAG/experiments/test_llama2_combined/output/hotpotqa_2025_03_15_17_20_naive',
 '/home/kucerj56/FlashRAG/experiments/test_llama2_combined/output/2wikimultihopqa_2025_03_15_17_54_naive',
 '/home/kucerj56/FlashRAG/experiments/test_llama2_combined/output/mmlu_2025_03_15_18_43_adaptive-rag',
 '/home/kucerj56/FlashRAG/experiments/test_llama2_combined/output/arc_2025_03_15_20_22_adaptive-rag',
 '/home/kucerj56/FlashRAG/experiments/test_llama2_combined/output/hotpotqa_2025_03_15_20_46_ad

In [ ]:
command = '/home/kucerj56/FlashRAG/.venv/bin/python /home/kucerj56/FlashRAG/examples/methods/run_eval_only.py  --config-file "{exp_dir}/config.yaml" --generated-dataset-path "{exp_dir}/generated.json"'
for dr in all_exp_dirs:
    print(f"echo ======================================================================================================")
    print(f"echo {dr}")
    print(f"cd {dr.split('output')[0]}")
    print(command.format(exp_dir=dr))

echo ======================================================================================================
echo /home/kucerj56/FlashRAG/experiments/test_llama2_combined/output/mmlu_2025_03_15_15_17_zero-shot
cd /home/kucerj56/FlashRAG/experiments/test_llama2_combined/
/home/kucerj56/FlashRAG/.venv/bin/python /home/kucerj56/FlashRAG/examples/methods/run_eval_only.py  --config-file "/home/kucerj56/FlashRAG/experiments/test_llama2_combined/output/mmlu_2025_03_15_15_17_zero-shot/config.yaml" --generated-dataset-path "/home/kucerj56/FlashRAG/experiments/test_llama2_combined/output/mmlu_2025_03_15_15_17_zero-shot/generated.json"
echo ======================================================================================================
echo /home/kucerj56/FlashRAG/experiments/test_llama2_combined/output/arc_2025_03_15_15_24_zero-shot
cd /home/kucerj56/FlashRAG/experiments/test_llama2_combined/
/home/kucerj56/FlashRAG/.venv/bin/python /home/kucerj56/FlashRAG/examples/methods/run_eval_only.py 

In [ ]:
import os
import yaml

def update_yaml_config_in_directories(directories):
    for dir_path in directories:
        config_file_path = os.path.join(dir_path, 'config.yaml')
        if os.path.isfile(config_file_path):
            try:
                # Load the YAML file
                with open(config_file_path, 'r') as file:
                    data = yaml.safe_load(file)

                # Ensure data is a dictionary
                if isinstance(data, dict):
                    # Update metric_setting
                    metric_setting = data.setdefault('metric_setting', {})
                    llm_judge_setting = metric_setting.setdefault('llm_judge_setting', {})
                    llm_judge_setting.setdefault('model_name', 'llama31-8B-instruct')
                    metric_setting.setdefault('retrieval_recall_topk', 5)
                    metric_setting.setdefault('tokenizer_name', 'gpt-4')

                    # Ensure we have the llm_judge_generator_override dictionary
                    llm_judge_generator_override = metric_setting.setdefault('llm_judge_generator_override', {})
                    # Explicitly set each key we need
                    llm_judge_generator_override['framework'] = 'vllm'
                    llm_judge_generator_override['generator_model'] = 'llama31-8B-instruct'
                    llm_judge_generator_override['generator_model_path'] = "/home/kucerj56/models/meta-llama/Llama-3.1-8B-Instruct"

                    # Update metrics list
                    metrics = data.get('metrics', [])
                    if 'llm_judge' in metrics:
                        metrics = [m if m != 'llm_judge' else 'llm_judge_matcher' for m in metrics]
                    data['metrics'] = metrics

                    # Write the updated data back to the file
                    with open(config_file_path, 'w') as file:
                        yaml.safe_dump(data, file, default_flow_style=False)

                    print(f"Updated config.yaml in {dir_path}")
                else:
                    print(f"Invalid YAML structure in {config_file_path}")

            except Exception as e:
                print(f"Error processing {config_file_path}: {e}")
        else:
            print(f"config.yaml not found in {dir_path}")

# Example usage
# all_exp_dirs = [
#     '/path/to/experiment1',
#     '/path/to/experiment2',
#     '/path/to/experiment3',
#     # Add more directories as needed
# ]

update_yaml_config_in_directories(all_exp_dirs)
# update_yaml_config_in_directories(["/home/kucerj56/FlashRAG/experiments/test_llama31_combined/output/mmlu_2025_03_15_12_57_naive"])

Updated config.yaml in /home/kucerj56/FlashRAG/experiments/test_llama2_combined/output/mmlu_2025_03_15_15_17_zero-shot
Updated config.yaml in /home/kucerj56/FlashRAG/experiments/test_llama2_combined/output/arc_2025_03_15_15_24_zero-shot
Updated config.yaml in /home/kucerj56/FlashRAG/experiments/test_llama2_combined/output/hotpotqa_2025_03_15_15_25_zero-shot
Updated config.yaml in /home/kucerj56/FlashRAG/experiments/test_llama2_combined/output/mmlu_2025_03_15_15_30_naive
Updated config.yaml in /home/kucerj56/FlashRAG/experiments/test_llama2_combined/output/arc_2025_03_15_16_58_naive
Updated config.yaml in /home/kucerj56/FlashRAG/experiments/test_llama2_combined/output/hotpotqa_2025_03_15_17_20_naive
Updated config.yaml in /home/kucerj56/FlashRAG/experiments/test_llama2_combined/output/2wikimultihopqa_2025_03_15_17_54_naive
Updated config.yaml in /home/kucerj56/FlashRAG/experiments/test_llama2_combined/output/mmlu_2025_03_15_18_43_adaptive-rag
Updated config.yaml in /home/kucerj56/FlashR